# Welcome to the Demo
Run the code block below to download the sample input file

In [ ]:
# Download Abundances.tsv from GitHub repo
!wget -q https://raw.githubusercontent.com/Rayyan-Tariq-Khan/ProfileScorer/main/Input/Abundances.tsv -O Abundances.tsv

# Confirm the file is present
import os
if os.path.exists("Abundances.tsv"):
    print("Abundances.tsv successfully downloaded and ready to use.")
else:
    print("Failed to download Abundances.tsv.")


# Single Profile Scorer
Run the cell below and enter Uniprot ID or the Gene name, and follow the instructions. (Example name  - P25705)

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from scipy.signal import find_peaks, peak_widths

# Load the TSV file
df = pd.read_csv("Abundances.tsv", sep="\t")

# Ask the user for the protein ID or gene name
query = input("Enter the UniProt ID or gene name of the protein: ").strip()

# Ask user whether to log-transform
log_input = input("Log-transform data? (y/n): ").strip().lower()
apply_log = log_input == 'y'

# Find the matching row (search in both columns)
match = df[(df.iloc[:, 0] == query) | (df.iloc[:, 1] == query)]

# Validate the match
if match.empty:
    print(f"No match found for '{query}'. Please check your input.")
else:
    # Extract raw abundance values
    raw_abundance = match.iloc[0, 2:].to_numpy(dtype=float)
    slices = np.arange(1, len(raw_abundance) + 1)

    # Detect peaks on raw data
    peaks, properties = find_peaks(raw_abundance, height=100000000, prominence=500000)

    # Calculate peak widths on raw data
    widths, width_heights, left_ips, right_ips = peak_widths(raw_abundance, peaks, rel_height=0.5)

    # Prepare display abundance (log-transformed if requested)
    display_abundance = np.log2(raw_abundance + 1) if apply_log else raw_abundance

    # Plot using Plotly
    fig = go.Figure()

    # Profile line
    fig.add_trace(go.Scatter(
        x=slices, y=display_abundance, mode='lines+markers', name='Abundance',
        line=dict(color='blue'), marker=dict(size=6)
    ))

    # Peaks
    fig.add_trace(go.Scatter(
        x=slices[peaks], y=display_abundance[peaks], mode='markers', name='Peaks',
        marker=dict(color='red', size=10, symbol='x')
    ))

    # Peak widths (still drawn using raw left/right indices, but using display_abundance height)
    for i, (left, right) in enumerate(zip(left_ips, right_ips)):
        # Use logged heights if applicable
        left_idx, right_idx = int(np.floor(left)), int(np.ceil(right))
        height_idx = peaks[i]
        height = display_abundance[height_idx]
        fig.add_trace(go.Scatter(
            x=[slices[left_idx], slices[right_idx]],
            y=[height, height],
            mode='lines',
            line=dict(color='green', dash='dash'),
            showlegend=(i == 0),
            name='Peak Width'
        ))

    # Layout
    fig.update_layout(
        title=f"Distribution Profile for {query} {'(Log2)' if apply_log else ''}",
        xaxis_title="BN-PAGE Slice Number",
        yaxis_title="Log2 Abundance" if apply_log else "Abundance",
        legend_title="Legend",
        height=500
    )

    fig.show()

    # Print peak summary
    print("\nPeak Summary:")
    print("-------------")
    print("Peak Count:", len(peaks))
    print("Peak Positions (Slice Numbers):", slices[peaks])
    print("Peak Heights (raw):", raw_abundance[peaks])
    print("Peak Widths (half-prominence, raw data):", widths)
    print("Peak Prominences (raw data):", properties['prominences'])


# Complex Verifier - Compare Multiple Profiles
Run the cell below and enter Uniprot IDs or the Gene names from a complex, and follow the instructions. (Example ATPase_hu complex - P25705, P06576, P36542, P30049, P56385, P56385, O75964, Q9UII2, P00846, P03928, Q96IX5, P24539, O75947, P48047)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.stats import pearsonr
from itertools import combinations

# Load data
df = pd.read_csv("Abundances.tsv", sep="\t")

# Ask user for protein names
input_names = input("Enter protein IDs or gene names separated by commas: ").split(",")
input_names = [name.strip() for name in input_names]

# Ask whether to use log2 y-axis scale
use_log2 = input("Display y-axis in binary log scale (log2)? [y/n]: ").lower().startswith("y")

# Find matching rows
matches = pd.DataFrame()
for name in input_names:
    match = df[(df.iloc[:, 0] == name) | (df.iloc[:, 1] == name)]
    if not match.empty:
        matches = pd.concat([matches, match])
    else:
        print(f"Protein '{name}' not found.")

if matches.empty:
    print("No valid proteins found. Exiting.")
    exit()

# Ask user which column to use for labeling
print("\nChoose which column to use for labeling the final plot:")
for i, col in enumerate(df.columns[:2]):
    print(f"{i}: {col}")
label_col_index = int(input("Enter column number (e.g. 0 or 1): "))
label_col = df.columns[label_col_index]

# Extract abundance profiles
profiles = {}
for _, row in matches.iterrows():
    label = row[label_col]
    abundance = row.iloc[2:].values.astype(float)
    profiles[label] = abundance

# Prepare plot
plt.figure(figsize=(12, 6))
slices = np.arange(1, len(next(iter(profiles.values()))) + 1)

# Plot all profiles with peaks
for label, abundance in profiles.items():
    peaks, _ = find_peaks(abundance, height=100000000, prominence=500000)
    plt.plot(slices, abundance, label=label, linestyle="-", marker="o")
    plt.scatter(slices[peaks], abundance[peaks], label=f"{label} Peaks", s=100, marker='x')

plt.xlabel("PAGE Slice Number")
plt.ylabel("Protein Abundance" + (" (log₂ scale)" if use_log2 else ""))
if use_log2:
    plt.yscale("log", base=2)
plt.title("Protein Complex Distribution Profiles")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

# Containers for group-wide similarity stats
cosine_similarities = []
pearson_correlations = []
jaccard_indices = []

# Pairwise comparisons
print("\n--- Pairwise Similarity Scores ---\n")
for (label1, profile1), (label2, profile2) in combinations(profiles.items(), 2):
    peaks1, _ = find_peaks(profile1, height=100000000, prominence=500000)
    peaks2, _ = find_peaks(profile2, height=100000000, prominence=500000)

    # Jaccard Index
    jaccard_index = len(set(peaks1) & set(peaks2)) / len(set(peaks1) | set(peaks2)) if len(set(peaks1) | set(peaks2)) > 0 else 0
    jaccard_indices.append(jaccard_index)

    peak_count_diff = abs(len(peaks1) - len(peaks2))

    common_peaks = sorted(set(peaks1) & set(peaks2))
    if len(common_peaks) > 1:
        profile1_common = [profile1[p] for p in common_peaks]
        profile2_common = [profile2[p] for p in common_peaks]
        peak_corr, _ = pearsonr(profile1_common, profile2_common)
    else:
        peak_corr = None

    try:
        pearson_corr, _ = pearsonr(profile1, profile2)
        pearson_correlations.append(pearson_corr)
    except ValueError:
        pearson_corr = None

    euclidean_dist = np.linalg.norm(profile1 - profile2)

    dot_product = np.dot(profile1, profile2)
    norm1 = np.linalg.norm(profile1)
    norm2 = np.linalg.norm(profile2)
    cosine_similarity = (dot_product / (norm1 * norm2)) if norm1 > 0 and norm2 > 0 else 0
    cosine_similarities.append(cosine_similarity)
    max_possible_dot = norm1 * norm2

    valid_scores = [jaccard_index, cosine_similarity]
    if pearson_corr is not None:
        valid_scores.append(pearson_corr)
    similarity_score = sum(valid_scores) / len(valid_scores)

    # Print results
    print(f"▶ {label1} vs {label2}")
    print(f"  Peak Jaccard Index: {jaccard_index:.2f}")
    print(f"  Peak Count Difference: {peak_count_diff}")
    print(f"  Peak Height Correlation: {peak_corr:.2f}" if peak_corr is not None else "  Peak Height Correlation: N/A")
    print(f"  Pearson Correlation (Shape): {pearson_corr:.2f}" if pearson_corr is not None else "  Pearson Correlation: N/A")
    print(f"  Euclidean Distance: {euclidean_dist:.2f}")
    print(f"  Dot Product: {dot_product:.2f}")
    print(f"  Theoretical Max Dot Product: {max_possible_dot:.2f}")
    print(f"  Cosine Similarity: {cosine_similarity:.2f}")
    print(f"  Overall Similarity Score: {similarity_score:.2f}\n")

# --- Group Similarity Score ---
if cosine_similarities and pearson_correlations and jaccard_indices:
    group_cosine = np.mean(cosine_similarities)
    group_pearson = np.mean(pearson_correlations)
    group_jaccard = np.mean(jaccard_indices)
    group_similarity_score = (group_cosine + group_pearson + group_jaccard) / 3

    print("\n=== Overall Group Similarity ===")
    print(f"Mean Cosine Similarity: {group_cosine:.2f}")
    print(f"Mean Pearson Correlation: {group_pearson:.2f}")
    print(f"Mean Jaccard Index: {group_jaccard:.2f}")
    print(f"→ Overall Group Similarity Score: {group_similarity_score:.2f} (0-1 scale)")
else:
    print("\nCould not compute group similarity score due to missing data.")
